# Default Probability Simulation

This notebook estimates the probability of bank defaults under two different models: a **coupled** system and an **independent** system. The simulation tracks the minimum asset value for each bank over time and counts a default when that minimum falls below a threshold.

## Table of Contents

- [Introduction](#introduction)
- [Parameters and Setup](#parameters-and-setup)
- [Simulation Dynamics](#simulation-dynamics)
- [Histogram and Results](#histogram-and-results)
- [Plotting the Results](#plotting-the-results)
- [Conclusion](#conclusion)

## Introduction

In this simulation, we model the dynamics of banks using a stochastic differential equation (SDE) with the Euler-Maruyama method. We simulate two cases:

1. **Coupled system** – In this model banks interact via the term `a * (mean_X - X)`, so that the evolution of each bank depends on the mean across banks.
2. **Independent system** – In this model the banks evolve independently based on random shocks.

For each simulation, we keep track of the running minimum for each bank, and count a default if the minimum ever goes below a predetermined level **D = -0.7**.

## Parameters and Setup

The following parameters are used in the simulation:

- **N:** 10  
  Number of banks. With 10 banks, default counts range from 0 to 10.
- **T:** 1.0  
  Total simulation time.
- **dt:** 0.001  
  Time step for the simulation.
- **steps:** Computed as `T / dt`  
  Total number of simulation steps.
- **a:** 10  
  Interaction parameter (used only in the coupled system).
- **sigma:** 1.0  
  Volatility representing the intensity of the random shocks.
- **D:** -0.7  
  Default threshold, where any bank whose running minimum falls below this value counts as having defaulted.
- **n_sim:** 10,000  
  Number of Monte Carlo simulation runs.

## Simulation Dynamics

The function `simulate_dynamics(coupled=True)` carries out the simulation for either the coupled or independent model. For each simulation:

1. **Initialization:**  
   Each bank’s asset value starts at zero, and the minimum value is also tracked from the start.

2. **Time Evolution:**  
   For each time step:
   - Compute the Brownian increment for each bank.
   - For the **coupled** case:  
     Update the bank’s value using the term `a * (mean_X - X)` where `mean_X` is the average across banks.
   - For the **independent** case:  
     The bank value is updated solely based on the noise (i.e., `sigma * dW`).
   - The running minimum is updated for each bank during the evolution.

3. **Default Counting:**  
   At the end of each simulation path, the number of banks that have defaulted (i.e. where the running minimum goes under D) is recorded.

## Histogram and Results

After running the simulations for both the coupled and independent systems, histograms are computed to represent the probability distribution of the number of defaults. The histograms are normalized so that the y-axis represents a probability.

- **x_vals:**  
  The number of defaults (from 0 to N).

- **hist_coupled:**  
  Normalized histogram for the coupled simulation.
  
- **hist_independent:**  
  Normalized histogram for the independent simulation.

## Plotting the Results

The final section of the notebook produces a plot with the following features:

- Plots the histogram for the coupled system using a solid blue line.
- Plots the histogram for the independent system using a dashed blue line.
- Axis labels for the number of defaults and their probability.
- Customizations for the x-axis ticks and limits to clearly display the 0 to N defaults.
- A legend to distinguish between the two simulation cases.

## Conclusion

This notebook provides a quantitative analysis of default probabilities by:
- Simulating bank asset values using SDEs.
- Comparing the default behavior under coupled versus independent dynamics.
- Visualizing the loss distribution to gain insights into systemic risk.

By varying parameters such as the interaction strength, volatility, and default threshold, further investigations can illuminate the factors contributing to systemic risk in banking systems.

In [9]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
N = 10       # Number of banks (set to 10 for discrete default counts up to 10)
T = 1.0
dt = 0.001
steps = int(T / dt)
a = 10
sigma = 1.0
D = -0.7
n_sim = 10_000

In [ ]:
def simulate_dynamics(coupled=True):
    """
    Simulate the dynamics of banks' assets and count defaults.
    :param coupled: If True, simulate coupled dynamics; if False, independent.
    :return: Array of default counts for each simulation.
    """
    defaults = np.zeros(n_sim, dtype=int)

    for sim in range(n_sim):
        X = np.zeros(N)
        min_X = np.zeros(N)  # Track minimum for each bank

        for t in range(steps):
            dW = np.random.normal(0, np.sqrt(dt), N)
            if coupled:
                mean_X = np.mean(X)
                dX = a * (mean_X - X) * dt + sigma * dW
                X += dX
                min_X = np.minimum(min_X, X)
                # In coupled case, you can still check final X
            else:
                dX = sigma * dW
                X += dX
                min_X = np.minimum(min_X, X)  # Track running minimum
        # Check for defaults
        defaults[sim] = np.sum(min_X < D)
        if sim % 1000 == 0:
            print(f"Simulation {sim}/{n_sim} completed.")
    return defaults


In [ ]:
# Simulate both coupled and independent systems
np.random.seed(42)
defaults_coupled = simulate_dynamics(coupled=True)
defaults_independent = simulate_dynamics(coupled=False)

# Compute histogram: number of defaults vs probability
max_defaults = N
bins = np.arange(0, max_defaults + 2) - 0.5  # Bin edges centered on integers
hist_coupled, _ = np.histogram(defaults_coupled, bins=bins, density=True)
hist_independent, _ = np.histogram(defaults_independent, bins=bins, density=True)
x_vals = np.arange(0, max_defaults + 1)


Simulation 0/10000 completed.
Simulation 1000/10000 completed.
Simulation 2000/10000 completed.
Simulation 3000/10000 completed.
Simulation 4000/10000 completed.
Simulation 5000/10000 completed.
Simulation 6000/10000 completed.
Simulation 7000/10000 completed.
Simulation 8000/10000 completed.
Simulation 9000/10000 completed.
Simulation 0/10000 completed.
Simulation 1000/10000 completed.
Simulation 2000/10000 completed.
Simulation 3000/10000 completed.
Simulation 4000/10000 completed.
Simulation 5000/10000 completed.
Simulation 6000/10000 completed.
Simulation 7000/10000 completed.
Simulation 8000/10000 completed.
Simulation 9000/10000 completed.


In [ ]:
# Plotting
#%matplotlib qt
plt.figure()
plt.plot(x_vals, hist_coupled, 'b-', label=f'Coupled (a={a})')
plt.plot(x_vals, hist_independent, 'b--', label='Independent')
plt.xlabel('# of defaults')
plt.ylabel('Probability of # of defaults')
#plt.title('Loss Distribution (Coupled vs Independent)')
plt.xlim(0, N)
plt.ylim(0, 1)
plt.xticks(np.arange(0, N+1, 2))
plt.legend()
#plt.grid(True)
#plt.save('images/Default_probability.png', dpi=300)
plt.show()